In [1]:
import pandas as pd
import cv2
import yt_dlp as youtube_dl
import os
from multiprocessing.pool import ThreadPool as Pool

In [2]:
youtube_link = 'http://youtube.com/watch?v='


In [3]:
# Load video csv
df = pd.read_csv('videos.csv')
df

,video_name,resolution,fps,yt_id
0,20221009-W-Ostrava-F-Iga_Swiatek-Barbora_Krejc...,1280x720,25.000000,Ym2XI1FeCxg
1,20211017-W-Indian_Wells-F-Victoria_Azarenka-Pa...,1280x720,25.000000,TPKtTzU-9rE
2,20210815-W-Montreal-F-Karolina_Pliskova-Camila...,1280x720,23.976024,Y4aQrXhbknU
3,20210731-W-Olympics-F-Belinda_Bencic-Marketa_V...,1280x720,25.000000,gGM9F18kk2M
4,20210401-W-Miami-SF-Bianca_Andreescu-Maria_Sak...,1280x720,25.000000,7u63Js0I9Lo
...,...,...,...,...
146,20210905-W-US_Open-R16-Leylah_Fernandez-Angeli...,1280x720,29.970030,8LJubnASt6s
147,20210903-W-US_Open-R32-Leylah_Fernandez-Naomi_...,1280x720,29.970030,2jw0D61Fle4
148,20180605-M-Roland_Garros-QF-Marco_Cecchinato-N...,1280x720,25.000000,BCjHPehrILw
149,20130607-M-Roland_Garros-SF-Novak_Djokovic-Raf...,1280x720,25.000000,sEiB2E_N3OA


In [4]:
# Create a mapping for each video name to its URL link
video_to_link_map = {}
for idx, row in df.iterrows():
    video_to_link_map[row['video_name']] = {'resolution': row['resolution'],
                                           'fps': row['fps'],
                                           'url': row['yt_id']}
video_to_link_map

{'20221009-W-Ostrava-F-Iga_Swiatek-Barbora_Krejcikova': {'resolution': '1280x720',
  'fps': 25.0,
  'url': 'Ym2XI1FeCxg'},
 '20211017-W-Indian_Wells-F-Victoria_Azarenka-Paula_Badosa': {'resolution': '1280x720',
  'fps': 25.0,
  'url': 'TPKtTzU-9rE'},
 '20210815-W-Montreal-F-Karolina_Pliskova-Camila_Giorgi': {'resolution': '1280x720',
  'fps': 23.976023976023978,
  'url': 'Y4aQrXhbknU'},
 '20210731-W-Olympics-F-Belinda_Bencic-Marketa_Vondrousova': {'resolution': '1280x720',
  'fps': 25.0,
  'url': 'gGM9F18kk2M'},
 '20210401-W-Miami-SF-Bianca_Andreescu-Maria_Sakkari': {'resolution': '1280x720',
  'fps': 25.0,
  'url': '7u63Js0I9Lo'},
 '20200218-W-Dubai-R32-Elena_Rybakina-Sofia_Kenin': {'resolution': '1280x720',
  'fps': 25.0,
  'url': 'r7mXCzipgBM'},
 '20200111-W-Brisbane-SF-Naomi_Osaka-Karolina_Pliskova': {'resolution': '1280x720',
  'fps': 25.0,
  'url': 'cZI2JfMF88I'},
 '20200110-W-Auckland-QF-Amanda_Anisimova-Eugenie_Bouchard': {'resolution': '1280x720',
  'fps': 25.0,
  'url': 'uaL3

In [5]:
train = pd.read_json('test.json')
train

,events,fps,height,width,num_frames,video,ply1_name,ply2_name,ply1_hand,ply2_hand,ply1_set,ply2_set,ply1_game,ply2_game,ply1_point,ply2_point
0,"[{'frame': 73, 'label': 'far_-_serve_B_in'}, {...",29,720,1280,410,20210909-W-US_Open-SF-Emma_Raducanu-Maria_Sakk...,Emma Raducanu,Maria Sakkari,RH,RH,0,0,0,0,0,1
1,"[{'frame': 50, 'label': 'far_-_serve_-_last'}]",29,720,1280,195,20210909-W-US_Open-SF-Emma_Raducanu-Maria_Sakk...,Emma Raducanu,Maria Sakkari,RH,RH,0,0,0,0,0,2
2,"[{'frame': 50, 'label': 'far_-_serve_W_in'}, {...",29,720,1280,279,20210909-W-US_Open-SF-Emma_Raducanu-Maria_Sakk...,Emma Raducanu,Maria Sakkari,RH,RH,0,0,0,0,1,3
3,"[{'frame': 93, 'label': 'far_-_serve_T_in'}, {...",29,720,1280,272,20210909-W-US_Open-SF-Emma_Raducanu-Maria_Sakk...,Emma Raducanu,Maria Sakkari,RH,RH,0,0,0,0,2,3
4,"[{'frame': 50, 'label': 'far_-_serve_-_last'}]",29,720,1280,164,20210909-W-US_Open-SF-Emma_Raducanu-Maria_Sakk...,Emma Raducanu,Maria Sakkari,RH,RH,0,0,0,0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,"[{'frame': 33, 'label': 'far_-_serve_W_in'}, {...",25,720,1280,120,20230125-W-Australian_Open-QF-Magda_Linette-Ka...,Karolina Pliskova,Magda Linette,RH,RH,0,1,5,5,0,1
2267,"[{'frame': 50, 'label': 'far_-_serve_W_in'}, {...",25,720,1280,203,20230125-W-Australian_Open-QF-Magda_Linette-Ka...,Karolina Pliskova,Magda Linette,RH,RH,0,1,5,6,1,0
2268,"[{'frame': 27, 'label': 'far_-_serve_W_in'}, {...",25,720,1280,149,20230125-W-Australian_Open-QF-Magda_Linette-Ka...,Karolina Pliskova,Magda Linette,RH,RH,0,1,5,6,1,1
2269,"[{'frame': 10, 'label': 'far_-_serve_W_in'}, {...",25,720,1280,139,20230125-W-Australian_Open-QF-Magda_Linette-Ka...,Karolina Pliskova,Magda Linette,RH,RH,0,1,5,6,1,2


In [6]:
def download_frame(url, start, end, frame_needed, name, timestamps, cls):
    # Function to download frames in each individual action, starting from 10 frames before the video up till the next event
    # url: link of video
    # start: starting frame of whole rally
    # end: ending frame of whole rally
    # frame_needed: frame number of event
    # timestamps: list of all frame numbers containing events in rally
    # cls: label of the video
    print(f"Starting on {frame_needed} on {name}")
    # Change path to your own path
    os.mkdir(f'OUTPUT_DIR/{name}_{frame_needed}')
    ydl_opts={}
    ydl=youtube_dl.YoutubeDL(ydl_opts)
    info_dict=ydl.extract_info(url, download=False)

    formats = info_dict.get('formats',None)
    for f in formats:
        if f.get('format_note',None) == '720p':
            url = f.get('url',None)
            cap = cv2.VideoCapture(url)
            idx = timestamps.index(frame_needed)
            new_start, new_end = (start + max(0, frame_needed - 10), start + timestamps[idx + 1])
            cap.set(1, new_start)
            for i in range(new_start, new_end + 1):
                ret, frame = cap.read()
                if not ret:
                    print('Frame cannot be found') 
                    return
                cv2.imwrite(f'OUTPUT_DIR/{name}_{frame_needed}/frame_{i-start}.jpg', frame)
            cap.release()


In [ ]:
# Count valid videos
valid_count = 0
# Count invalid videos (e.g. video no longer exists)
invalid_count = 0

# Multithreading to speed up download, adjust as per requirement
pool_size = 3
pool = Pool(pool_size)

for index, row in train.iterrows():
    events = row['events']
    name = row['video']
    segmented_vid = name.split('_')
    # get start frame
    start = int(segmented_vid[-2])
    end = int(segmented_vid[-1])
    timestamps = []
    for event in events:
        timestamps.append(int(event['frame']))
    timestamps.append(end - start)
    for event in events:
        label = event['label']
        frame = int(event['frame'])
        try:
            url = youtube_link + video_to_link_map['_'.join(segmented_vid[:-2])]['url']
            pool.apply_async(download_frame, (url, start, end, frame, name, timestamps, label))
            valid_count += 1
        except:
            invalid_count += 1
    

pool.close()
pool.join()

print(f'{valid_count} valid data points found!')
print(f'{invalid_count} invalid data points found!')

In [ ]:
# delete all invalid video folders
def delete_empty_folders(root):

    deleted = set()
    
    for current_dir, subdirs, files in os.walk(root, topdown=False):

        still_has_subdirs = False
        for subdir in subdirs:
            if os.path.join(current_dir, subdir) not in deleted:
                still_has_subdirs = True
                break
    
        if not any(files) and not still_has_subdirs:
            os.rmdir(current_dir)
            deleted.add(current_dir)

    return len(deleted)

delete_empty_folders('OUTPUT_DIR')

# Convert frames into mp4

In [ ]:
# set up directory for all classes
classes = {'start',
'end',
'far_-_serve_-_last',
'far_-_serve_T_in',
'far_-_serve_B_in',
'far_-_serve_W_in',
'far_bh_return_-_last',
'far_bh_return_CC_in',
'far_bh_return_DL_in',
'far_bh_return_DM_in',
'far_bh_return_II_in',
'far_bh_return_IO_in',
'far_fh_return_-_last',
'far_fh_return_CC_in',
'far_fh_return_DL_in',
'far_fh_return_DM_in',
'far_fh_return_II_in',
'far_fh_return_IO_in',
'far_bh_stroke_-_last',
'far_bh_stroke_CC_in',
'far_bh_stroke_DL_in',
'far_bh_stroke_DM_in',
'far_bh_stroke_II_in',
'far_bh_stroke_IO_in',
'far_fh_stroke_-_last',
'far_fh_stroke_CC_in',
'far_fh_stroke_DL_in',
'far_fh_stroke_DM_in',
'far_fh_stroke_II_in',
'far_fh_stroke_IO_in',
'near_-_serve_-_last',
'near_-_serve_T_in',
'near_-_serve_B_in',
'near_-_serve_W_in',
'near_bh_return_-_last',
'near_bh_return_CC_in',
'near_bh_return_DL_in',
'near_bh_return_DM_in',
'near_bh_return_II_in',
'near_bh_return_IO_in',
'near_fh_return_-_last',
'near_fh_return_CC_in',
'near_fh_return_DL_in',
'near_fh_return_DM_in',
'near_fh_return_II_in',
'near_fh_return_IO_in',
'near_bh_stroke_-_last',
'near_bh_stroke_CC_in',
'near_bh_stroke_DL_in',
'near_bh_stroke_DM_in',
'near_bh_stroke_II_in',
'near_bh_stroke_IO_in',
'near_fh_stroke_-_last',
'near_fh_stroke_CC_in',
'near_fh_stroke_DL_in',
'near_fh_stroke_DM_in',
'near_fh_stroke_II_in',
'near_fh_stroke_IO_in'
}

for cls in classes: 
    os.mkdir(f'DATA_DIR/{cls}')

In [7]:
def save(dir, name, frame, cls):
    # Chains frames together and saves in a directory
    # dir: dir containing frames
    # name: name of rally
    # frame: frame of event
    # cls: label of event
    start = max(frame - 10, 0)
    empty = True
    # Change output dir accordingly
    os.system(f"ffmpeg -r 25 -start_number {start} -i {dir}/frame_%d.jpg -vcodec mpeg4 -y OUTPUT/{cls}/{name}_{frame}.mp4")

    print(f"Done: {name}_{frame}" )

In [ ]:
valid = 0
invalid = 0

# Multithreading to speed up download, adjust as per requirement
pool_size = 4
pool = Pool(pool_size)

for index, row in train.iterrows():
    try:
        events = row['events']
        name = row['video']
        for event in events:
            label = event['label']
            segmented_vid = name.split('_')
            # get start frame
            frame = int(event['frame'])
            pool.apply_async(save(f'DATA_DIR/{name}_{frame}', name, frame, label))
        valid += 1
    except:
        invalid += 1
        
pool.close()
pool.join
print(valid, invalid)